In [1]:
import math
from math import log, sqrt, exp
import numpy as np
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import brentq

import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore') 

In [2]:
SPX_df = pd.read_csv("SPX_options.csv")
SPY_df = pd.read_csv("SPY_options.csv")
rates_df = pd.read_csv("zero_rates_20201201.csv")

In [3]:
SPX_spot=3662.45
SPY_spot=366.02

# Use at the money strike to get option volatility
SPX_strike=3662.45
SPY_strike=366.02

expiry_date = pd.to_datetime("20210115", format="%Y%m%d")

# Functions

In [5]:
def h_func(x):
    return x ** (1/3) + (3/2) * np.log(x) +10.0

In [6]:
def h_prime(x):
    return (1/3) * (x**(-2/3)) + (3/2) * (1/x)

In [7]:
def h_prime_prime(x):
    return (-2/9) * (x**(-5/3)) + (3/2) * (-1/(x**2))

In [8]:
def calculate_black_scholes_d1_d2(S: float, K: float, r: float, T: float, sigma: float) -> tuple[float, float]:
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

In [9]:
def black_scholes_vanilla_call(S: float, K: float, r: float, T: float, sigma: float) -> float:
    d1, d2 = calculate_black_scholes_d1_d2(S, K, r, T, sigma)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [10]:
def black_scholes_vanilla_put(S: float, K: float, r: float, T: float, sigma: float) -> float:
    d1, d2 = calculate_black_scholes_d1_d2(S, K, r, T, sigma)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [11]:
def black_scholes_implied_volatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                black_scholes_vanilla_call(S, K, r, T, x),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                black_scholes_vanilla_put(S, K, r, T, x),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [12]:
def calculate_bachelier_d(S: float, K: float, T: float, sigma: float) -> float:
    return (S - K) / (sigma * np.sqrt(T))

In [13]:
def bachelier_vanilla_call(S: float, K: float, r: float, T: float, sigma: float) -> float:
    sigma_adjusted = sigma * S
    d = calculate_bachelier_d(S, K, T, sigma_adjusted)
    return np.exp(-r * T) * ((S - K) * norm.cdf(d) + sigma_adjusted * np.sqrt(T) * norm.pdf(d))

In [14]:
def bachelier_vanilla_put(S: float, K: float, r: float, T: float, sigma: float) -> float:
    sigma_adjusted = sigma * S
    d = calculate_bachelier_d(S, K, T, sigma_adjusted)
    return np.exp(-r * T) *((K - S) * norm.cdf(-d) + sigma_adjusted * np.sqrt(T) * norm.pdf(d))

In [15]:
def bachelier_implied_volatility(S, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                 bachelier_vanilla_call(S, K, r, T, x),
                                1e-12, 10.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                bachelier_vanilla_call(S, K, r, T, x),
                                1e-12, 10.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol

# Prepare data for rates

In [17]:
rates_df["date"] = pd.to_datetime(rates_df["date"], format="%Y%m%d")
rates_df["rate_decimal"] = rates_df["rate"] / 100
rates_df = rates_df.drop(["date"], axis=1)
rates_df.set_index("days", inplace=True)
rates_df = rates_df.reindex(np.arange(rates_df.index.min(), rates_df.index.max() + 1))
rates_df = rates_df.interpolate(method="linear")

# Prepare data for SPX

In [19]:
SPX_df["date"] = pd.to_datetime(SPX_df["date"], format="%Y%m%d")
SPX_df["exdate"] = pd.to_datetime(SPX_df["exdate"], format="%Y%m%d")
SPX_df["days_to_expiry"] = (SPX_df["exdate"] - SPX_df["date"]) / pd.Timedelta(days=1)
SPX_df["years_to_expiry"] = SPX_df["days_to_expiry"] / 365
SPX_df["mid_price"] = 0.5 * (SPX_df["best_bid"] + SPX_df["best_offer"])
SPX_df["strike_price"] = SPX_df["strike_price"] / 1000
SPX_df["options_type"] = SPX_df["cp_flag"].map(lambda x: "call" if x == "C" else "put")
SPX_df=SPX_df.merge(rates_df,
                    left_on="days_to_expiry",
                    right_index=True)
# Filter the dataframe to get only exdates which we want
SPX_df = SPX_df [SPX_df ["exdate"] == expiry_date].reset_index(drop=True)

# impl market volatility column
SPX_df["vols"] = SPX_df.apply(
    lambda x: black_scholes_implied_volatility(
        SPX_spot,
        x["strike_price"],
        x["rate_decimal"],
        x["mid_price"],
        x["years_to_expiry"],
        x["options_type"]
    ),
    axis=1,
)
SPX_df.dropna(inplace=True)
SPX_df.reset_index(drop=True,inplace=True)

# Prepare data for SPY

In [21]:
SPY_df["date"] = pd.to_datetime(SPY_df["date"], format="%Y%m%d")
SPY_df["exdate"] = pd.to_datetime(SPY_df["exdate"], format="%Y%m%d")
SPY_df["days_to_expiry"] = (SPY_df["exdate"] - SPY_df["date"]) / pd.Timedelta(days=1)
SPY_df["years_to_expiry"] = SPY_df["days_to_expiry"] / 365
SPY_df["mid_price"] = 0.5 * (SPY_df["best_bid"] + SPY_df["best_offer"])
SPY_df["strike_price"] = SPY_df["strike_price"] / 1000
SPY_df["options_type"] = SPY_df["cp_flag"].map(lambda x: "call" if x == "C" else "put")

SPY_df=SPY_df.merge(rates_df,
                    left_on="days_to_expiry",
                    right_index=True)

# Filter the dataframe to get only exdates which we want
SPY_df = SPY_df [SPY_df ["exdate"] == expiry_date].reset_index(drop=True)

# impl market volatility column
SPY_df["vols"] = SPY_df.apply(
    lambda x: black_scholes_implied_volatility(
        SPY_spot,
        x["strike_price"],
        x["rate_decimal"],
        x["mid_price"],
        x["years_to_expiry"],
        x["options_type"]
    ),
    axis=1,
)
SPY_df.dropna(inplace=True)
SPY_df.reset_index(drop=True,inplace=True)

# Black-Scholes Sigma SPX

In [23]:
SPX_ATM = np.interp(SPX_strike, 
                    SPX_df["strike_price"], 
                    SPX_df["mid_price"])

In [24]:
SPX_BS_call_sigma = black_scholes_implied_volatility(SPX_spot,
                                                     SPX_strike,
                                                     SPX_df["rate_decimal"][0],
                                                     SPX_ATM,
                                                     SPX_df["years_to_expiry"][0],
                                                     payoff = 'call')


SPX_BS_put_sigma = black_scholes_implied_volatility(SPX_spot,
                                                    SPX_strike,
                                                    SPX_df["rate_decimal"][0],
                                                    SPX_ATM,
                                                    SPX_df["years_to_expiry"][0],
                                                    payoff = 'put')

SPX_ATM_sigma = (SPX_BS_call_sigma + SPX_BS_put_sigma)/2

print(f"ATM sigma for SPX using the Black-Scholes model is {SPX_ATM_sigma:.5f}")

ATM sigma for SPX using the Black-Scholes model is 0.18748


# Bachelier Sigma SPX

In [26]:
# Get bachelier sigma from the Black-Scholes price of option to get the Bachelier model volatility equivalent.

SPX_bachelier_sigma = (SPX_ATM * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])) / (SPX_spot * np.sqrt(SPX_df["years_to_expiry"][0] / (2 * np.pi)))

print(f"ATM sigma for SPX using the Bachelier model is {SPX_bachelier_sigma :.5f}")

ATM sigma for SPX using the Bachelier model is 0.18747


In [27]:
SPX_Bachelier_call_sigma = bachelier_implied_volatility(SPX_spot,
                                                        SPX_strike,
                                                        SPX_df["rate_decimal"][0],
                                                        SPX_ATM,
                                                        SPX_df["years_to_expiry"][0],
                                                        payoff = 'call')


SPX_Bachelier_put_sigma = bachelier_implied_volatility(SPX_spot,
                                                       SPX_strike,
                                                       SPX_df["rate_decimal"][0],
                                                       SPX_ATM,
                                                       SPX_df["years_to_expiry"][0],
                                                       payoff = 'put')

SPX_Bachelier_sigma = (SPX_Bachelier_call_sigma + SPX_Bachelier_put_sigma)/2

print(f"ATM sigma for SPX using the Bachelier model is {SPX_Bachelier_sigma:.5f}")

ATM sigma for SPX using the Bachelier model is 0.18747


# Black-Scholes Sigma for SPY

In [29]:
# Find price of ATM option
SPY_ATM = np.interp(SPY_strike, 
                    SPY_df["strike_price"], 
                    SPY_df["mid_price"])

In [30]:
SPY_BS_call_sigma = black_scholes_implied_volatility(SPY_spot,
                                                     SPY_strike,
                                                     SPY_df["rate_decimal"][0],
                                                     SPY_ATM,
                                                     SPY_df["years_to_expiry"][0],
                                                     payoff = 'call')


SPY_BS_put_sigma = black_scholes_implied_volatility(SPY_spot,
                                                    SPY_strike,
                                                    SPY_df["rate_decimal"][0],
                                                    SPY_ATM,
                                                    SPY_df["years_to_expiry"][0],
                                                    payoff = 'put')

SPY_ATM_sigma = (SPY_BS_call_sigma + SPY_BS_put_sigma)/2

print(f"ATM sigma for SPY using the Black-Scholes model is {SPY_ATM_sigma:.5f}")

ATM sigma for SPY using the Black-Scholes model is 0.19684


# Bachelier Sigma for SPY

In [32]:
# Get bachelier sigma from interpolated price of the ATM option
SPY_bachelier_sigma = (SPY_ATM * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])) / (SPY_spot * np.sqrt(SPY_df["years_to_expiry"][0] / (2 * np.pi)))

print(f"ATM sigma for SPY using the Bachelier model is {SPY_bachelier_sigma :.5f}")

ATM sigma for SPY using the Bachelier model is 0.19682


In [33]:
SPY_Bachelier_call_sigma = bachelier_implied_volatility(SPY_spot,
                                                        SPY_strike,
                                                        SPY_df["rate_decimal"][0],
                                                        SPY_ATM,
                                                        SPY_df["years_to_expiry"][0],
                                                        payoff = 'call')


SPY_Bachelier_put_sigma = bachelier_implied_volatility(SPY_spot,
                                                       SPY_strike,
                                                       SPY_df["rate_decimal"][0],
                                                       SPY_ATM,
                                                       SPY_df["years_to_expiry"][0],
                                                       payoff = 'put')

SPY_Bachelier_sigma = (SPY_Bachelier_call_sigma + SPY_Bachelier_put_sigma)/2

print(f"ATM sigma for SPY using the Bachelier model is {SPY_Bachelier_sigma:.5f}")

ATM sigma for SPY using the Bachelier model is 0.19682


# Exotic No.1 : Black-Scholes

Expected Black Scholes payoff is defined as

$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{9}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

The price is

$$
V_0 = e^{-rT}E[V_T]
$$

In [35]:
def black_scholes_price(S, rate, sigma, T):
    return np.exp(-rate * T) * (
        (
            np.power(S, 1.0 / 3.0)
            * np.exp((rate - 0.5 * sigma**2) * T * (1.0 / 3.0))
            * np.exp(0.5 * (1.0 / 9.0) * T * sigma**2)
            + 1.5 * (np.log(S) + (rate - 0.5 * sigma**2) * T)
            + 10
        )
    )

In [36]:
SPX_BS_price = black_scholes_price(SPX_spot, SPX_df["rate_decimal"][0], SPX_ATM_sigma, SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Black-Scholes model with SPX as the underlying is {SPX_BS_price:5f}")

Price of the Exotic option using the Black-Scholes model with SPX as the underlying is 37.704607


In [37]:
SPY_BS_price = black_scholes_price(SPY_spot, SPY_df["rate_decimal"][0], SPY_ATM_sigma, SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Black-Scholes model with SPY as the underlying is {SPY_BS_price:5f}")

Price of the Exotic option using the Black-Scholes model with SPY as the underlying is 25.994282


# Exotic No.1 : Bachelier

Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 \sqrt{T} x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 \sqrt{T} x) e^\frac{-x^2}{2}\,dx
+10
$$

Note $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [39]:
def integrand_1(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * np.power((S + sigma * S * np.sqrt(T) * x), 1.0 / 3.0)
        * np.exp(-0.5 * np.power(x, 2))
    )

In [40]:
def integrand_2(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * 1.5
        * np.log(S + sigma * S * np.sqrt(T) * x)
        * np.exp(-0.5 * np.power(x, 2))
    )

In [41]:
def bachelier_price(S, rate, sigma, T):
    lower_bound = -1 / (sigma * np.sqrt(T))
    I_1 = quad(lambda x: integrand_1(x, S, sigma, T), lower_bound, np.inf)
    I_2 = quad(lambda x: integrand_2(x, S, sigma, T), lower_bound, np.inf)
    V_0_bachelier = np.exp(-rate * T) * (I_1[0] + I_2[0] + 10)
    return V_0_bachelier

In [42]:
SPX_Bachelier_price = bachelier_price(SPX_spot, SPX_df["rate_decimal"][0], SPX_ATM_sigma, SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Bachelier model with SPX as the underlying is {SPX_Bachelier_price:5f}")

Price of the Exotic option using the Bachelier model with SPX as the underlying is 37.702870


In [43]:
SPY_Bachelier_price = bachelier_price(SPY_spot, 
                                      SPY_df["rate_decimal"][0], 
                                      SPY_ATM_sigma, 
                                      SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the Bachelier model with SPY as the underlying is {SPY_Bachelier_price:5f}")

Price of the Exotic option using the Bachelier model with SPY as the underlying is 25.993253


# Exotic No.1 : SABR

For SABR payoff, we must retrieve the volatility using previous calibration result

$$
h(S_T) = S_T^{\frac{1}{3}} + 1.5 log(S_T) + 10 \\
h''(S_T) = -\frac{2}{9}S_T^{-\frac{5}{3}} - 1.5\frac{1}{S_T^2} \\
F = S_0 e ^ {rT}
$$

The price is

$$
V_0 = e^{-rT}h(F) + \int_{0}^{F} h''(K)P(K) \,dK + \int_{F}^{\infty} h''(K)C(K) \,dK
$$


### For SPX, from Part 2

'45': {'F': 3663.3762493669747,
  'alpha': 1.8165044322089068,
  'beta': 0.7,
  'rho': -0.4043017551686191,
  'nu': 2.790158328184338},

### For SPY, from Part 2

'45': {'F': 366.11256803322914,
  'alpha': 0.9081326349323565,
  'beta': 0.7,
  'rho': -0.4887794472282475,
  'nu': 2.72851634129598},


In [47]:
SPX_alpha = 1.8165044322089068
SPX_beta = 0.7
SPX_rho = -0.4043017551686191
SPX_nu = 2.790158328184338

In [48]:
SPY_alpha = 0.9081326349323565
SPY_beta = 0.7
SPY_rho = -0.4887794472282475
SPY_nu = 2.72851634129598

In [49]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [50]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return black_scholes_vanilla_call(S, K, r, T, sabr_vol)

In [51]:
def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return black_scholes_vanilla_put(S, K, r, T, sabr_vol)

In [52]:
def sabr_price(S, K, r, alpha, beta, rho, nu, T):
    F = S * np.exp(r * T)

    I_put = quad(
        lambda x: h_prime_prime(x) * SABRPut(S, K, r, alpha, beta, rho, nu, T),
        0,
        F,
    )

    I_call = quad(
        lambda x: h_prime_prime(x) * SABRCall(S, K, r, alpha, beta, rho, nu, T),
        F,
        np.inf,
    )
    V_0_SABR = np.exp(-r * T) * h_func(F) + I_put[0] + I_call[0]
    return V_0_SABR

In [53]:
SPX_SABR_price = sabr_price(SPX_spot, 
                            SPX_strike,
                            SPX_df["rate_decimal"][0], 
                            SPX_alpha, 
                            SPX_beta, 
                            SPX_rho, 
                            SPX_nu, 
                            SPX_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the SABR model with SPX as the underlying is {SPX_SABR_price:5f}")

Price of the Exotic option using the SABR model with SPX as the underlying is 37.713597


In [54]:
SPY_SABR_price = sabr_price(SPY_spot, 
                            SPY_strike,
                            SPY_df["rate_decimal"][0], 
                            SPY_alpha, 
                            SPY_beta, 
                            SPY_rho, 
                            SPY_nu, 
                            SPY_df["years_to_expiry"][0])

print(f"Price of the Exotic option using the SABR model with SPY as the underlying is {SPY_SABR_price:5f}")

Price of the Exotic option using the SABR model with SPY as the underlying is 26.000677


# Exotic No.2 : Black Scholes

$$
E\Biggl[\int_{0}^{T} \sigma_t^2 \,dt\Biggr] = 2e^{rT}\Biggl(\int_{0}^{F} \frac{P(K)}{K^2} \,dK + \int_{F}^{\infty} \frac{C(K)}{K^2} \,dK\Biggr)
$$

In [56]:
def black_scholes_call_integrand(S, K, r, sigma, T):
    price = black_scholes_vanilla_call(S, K, r, T, sigma) / K**2
    return price

In [57]:
def black_scholes_put_integrand(S, K, r, sigma, T):
    price = black_scholes_vanilla_put(S, K, r, T, sigma) / K**2
    return price

In [58]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: black_scholes_call_integrand(SPX_spot,
                                                                  x,
                                                                  SPX_df["rate_decimal"][0],
                                                                  SPX_ATM_sigma,
                                                                  SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: black_scholes_put_integrand(SPX_spot,
                                                                x,
                                                                SPX_df["rate_decimal"][0],
                                                                SPX_ATM_sigma,
                                                                SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_BS_expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the Black Scholes model is {SPX_BS_expected_variance:.5f}")

The integrated variance for SPX using the Black Scholes model is 0.00433


In [59]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: black_scholes_call_integrand(SPY_spot,
                                                                  x,
                                                                  SPY_df["rate_decimal"][0],
                                                                  SPY_ATM_sigma,
                                                                  SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: black_scholes_put_integrand(SPY_spot,
                                                                x,
                                                                SPY_df["rate_decimal"][0],
                                                                SPY_ATM_sigma,
                                                                SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_BS_Expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the Black Scholes model is {SPY_BS_Expected_variance:.5f}")

The integrated variance for SPY using the Black Scholes model is 0.00478


# Exotic No.2 : Bachelier

In [61]:
def bachelier_call_integrand(S, K, r, sigma, T):
    bachmodel = bachelier_vanilla_call(S, K, r, T, sigma)
    return bachmodel / K**2

In [62]:
def bachelier_put_integrand(S, K, r, sigma, T):
    bachmodel = bachelier_vanilla_put(S, K, r, T, sigma)
    return bachmodel / K**2

In [63]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: bachelier_call_integrand(SPX_spot,
                                                              x,
                                                              SPX_df["rate_decimal"][0],
                                                              SPX_bachelier_sigma,
                                                              SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: bachelier_put_integrand(SPX_spot,
                                                            x,
                                                            SPX_df["rate_decimal"][0],
                                                            SPX_bachelier_sigma,
                                                            SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_bachelier_expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the Bachelier model is {SPX_bachelier_expected_variance:.5f}")

The integrated variance for SPX using the Bachelier model is 0.00436


In [64]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: bachelier_call_integrand(SPY_spot,
                                                             x,
                                                             SPY_df["rate_decimal"][0],
                                                             SPY_bachelier_sigma,
                                                             SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: bachelier_put_integrand(SPY_spot,
                                                            x,
                                                            SPY_df["rate_decimal"][0],
                                                            SPY_bachelier_sigma,
                                                            SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_bachelier_expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the Bachelier model is {SPY_bachelier_expected_variance:.5f}")

The integrated variance for SPY using the Bachelier model is 0.00481


# Exotic No.2 : SABR

In [66]:
def sabr_call_integrand(S, K, r, alpha, beta, rho, nu, T):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [67]:
def sabr_put_integrand(S, K, r, alpha, beta, rho, nu, T):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [68]:
SPX_forward = SPX_spot * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0])

SPX_call_integrated = quad(lambda x: sabr_call_integrand(SPX_spot,
                                                         x,
                                                         SPX_df["rate_decimal"][0],
                                                         SPX_alpha,
                                                         SPX_beta,
                                                         SPX_rho,
                                                         SPX_nu,
                                                         SPX_df["years_to_expiry"][0]), SPX_forward, np.inf)

SPX_put_integrated = quad(lambda x: sabr_put_integrand(SPX_spot,
                                                       x,
                                                       SPX_df["rate_decimal"][0],
                                                       SPX_alpha,
                                                       SPX_beta,
                                                       SPX_rho,
                                                       SPX_nu,
                                                       SPX_df["years_to_expiry"][0]), 0.0, SPX_forward)

SPX_SABR_expected_variance = 2 * np.exp(SPX_df["rate_decimal"][0] * SPX_df["years_to_expiry"][0]) * (SPX_put_integrated[0] + SPX_call_integrated[0])

print(f"The integrated variance for SPX using the SABR model is {SPX_SABR_expected_variance:.5f}")

The integrated variance for SPX using the SABR model is 0.00635


In [69]:
SPY_forward = SPY_spot * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0])

SPY_call_integrated = quad(lambda x: sabr_call_integrand(SPY_spot,
                                                         x,
                                                         SPY_df["rate_decimal"][0],
                                                         SPY_alpha,
                                                         SPY_beta,
                                                         SPY_rho,
                                                         SPY_nu,
                                                         SPY_df["years_to_expiry"][0]), SPY_forward, np.inf)

SPY_put_integrated = quad(lambda x: sabr_put_integrand(SPY_spot,
                                                        x,
                                                        SPY_df["rate_decimal"][0],
                                                        SPY_alpha,
                                                        SPY_beta,
                                                        SPY_rho,
                                                        SPY_nu,
                                                        SPY_df["years_to_expiry"][0]), 0.0, SPY_forward)

SPY_SABR_expected_variance = 2 * np.exp(SPY_df["rate_decimal"][0] * SPY_df["years_to_expiry"][0]) * (SPY_put_integrated[0] + SPY_call_integrated[0])

print(f"The integrated variance for SPY using the SABR model is {SPY_SABR_expected_variance:.5f}")

The integrated variance for SPY using the SABR model is 0.00602
